# Объединение датафреймов

In [1]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [70]:
#ratings.loc[ratings.userId == 15]

In [3]:
aggregations = {
    'timestamp': ['min', 'max'],    
    'rating': 'count'
}
ratings = ratings.groupby('userId').agg(aggregations)

In [4]:
ratings_over100 = ratings[ratings['rating']['count'] > 100].reset_index()
ratings_over100

userId   timestamp             rating
                   min         max  count
0        4   949778714   949982274    204
1        8  1154389340  1154474527    116
2       15   997937239  1469330735   1700
3       17  1127468587  1127476640    363
4       19   855190091   855195373    423
..     ...         ...         ...    ...
253    656   986240991   986244044    128
254    659   834598040   866207451    142
255    664  1343731283  1441911722    519
256    665   992836298  1046967769    434
257    671  1063500751  1074784735    115

[258 rows x 4 columns]

In [5]:
def get_TTL(row):
    return str(int(row['timestamp']['max']) - int(row['timestamp']['min']))    

ratings_over100['TTLmean'] = ratings_over100.apply(get_TTL, axis = 1)
ratings_over100

userId   timestamp             rating    TTLmean
                   min         max  count           
0        4   949778714   949982274    204     203560
1        8  1154389340  1154474527    116      85187
2       15   997937239  1469330735   1700  471393496
3       17  1127468587  1127476640    363       8053
4       19   855190091   855195373    423       5282
..     ...         ...         ...    ...        ...
253    656   986240991   986244044    128       3053
254    659   834598040   866207451    142   31609411
255    664  1343731283  1441911722    519   98180439
256    665   992836298  1046967769    434   54131471
257    671  1063500751  1074784735    115   11283984

[258 rows x 5 columns]

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [7]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


In [8]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


In [9]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


In [10]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [13]:
rzd_grouped = rzd.groupby('client_id').sum()
auto_grouped = auto.groupby('client_id').sum()
air_grouped = air.groupby('client_id').sum()
revenue_merge = pd.merge(rzd_grouped, auto_grouped, on = 'client_id', how = 'outer')
revenue_merge = pd.merge(revenue_merge, air_grouped, on = 'client_id', how = 'outer')
revenue_merge

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [12]:
pd.merge(revenue_merge, client_base, on = 'client_id', how = 'outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 
Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

### Данные со слайдов

In [77]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [78]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [79]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [80]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [81]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [82]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [83]:
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [84]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [ ]:
visits_grouped.join(purchases_pivot, how='right')

### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [ ]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

In [ ]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

In [ ]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [ ]:
visits_grouped.join(purchases_pivot, how='inner')

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, how='outer')

### Дубликаты при объединении таблиц

In [ ]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

In [ ]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

In [ ]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

In [ ]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

In [ ]:
ratings.merge(movies, how = 'right', on = 'movieId')

### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [ ]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

In [ ]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

### Оптимизация хранения данных

In [ ]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [ ]:
joined.head()

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
len(ratings) == len(joined)

In [ ]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [ ]:
import numpy as np

In [ ]:
genres = ['Drama', 'Action', 'Thriller']

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base